# Analyze Accuracy of Article Scrape Output

#### Scrapy Structure/Lingo:
**Spiders** extract data **items**, which Scrapy send one by one to a configured **item pipeline** (if there is possible) to do post-processing on the items.)

## Import relevant packages...

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.response import open_in_browser
from scrapy.selector import Selector
from scrapy.pipelines.files import FilesPipeline

import json
import hashlib
import random
from tqdm import tqdm

## Helper functions...

## Initial variables...

In [2]:
class PDFItem(scrapy.Item):
    file_urls = scrapy.Field()
    
class MyFilesPipeline(FilesPipeline):

    def file_path(self, request, response=None, info=None):
        return '/full/' + request.meta.get('filename','')

    def get_media_requests(self, item, info):
        file_url = item['file_urls'][0]
        meta = {'filename': hashlib.sha1(file_url.encode('utf-8')).hexdigest()[:250]+ '.pdf'}
        yield scrapy.Request(url=file_url, meta=meta)
        
text = ''

## Define spider...

In [3]:
class accuracySpider(scrapy.Spider):
    name = 'accuracyspider'
    custom_settings = {'LOG_LEVEL': 'WARNING',
                      'ITEM_PIPELINES': {'__main__.MyFilesPipeline': 1},
            'FILES_STORE' : '/Users/jordangunn/newsrepo'}
    
    def start_requests(self):
        f = open('/Volumes/Elements/decembercycle/articles/cleanedarticles4.jl')
        counter = 0
        for line in tqdm(f):
            
            # An important filter that i might have to change

            # this too, # of articles to scrape
            counter = counter + 1
            if counter > 1000:
                break
            article = json.loads(line)
            yield scrapy.Request(article['link'],callback=self.downloadPDF, dont_filter=True)
        
    def downloadPDF(self, response):
        global text
        pdf = PDFItem()
        
        # strategy if article is provided as text
        readableContent = Selector(response).xpath('//div[@id="readableContent"]//text()').extract()
        if len(readableContent) > 0:
            with open('/Users/jordangunn/newsrepo/full/' + hashlib.sha1(response.url.encode('utf-8')).hexdigest()[:250] + '.txt', 'w') as f:
                f.write(''.join(readableContent))
                
        # supposing this failed, it must be a pdf and we'll download that
        else:
            try:
                pdf['file_urls'] = [Selector(response).xpath('//*[text()="Open with your PDF reader"]/@href').extract()[0]]
            except IndexError:
                try:
                    pdf['file_urls'] = [Selector(response).xpath("//a[@download='ProQuestDocument.pdf']/@href").extract()[0]]
                except IndexError:
                    print('IndexError')
            yield pdf

In [4]:
process = CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'})

process.crawl(accuracySpider)
process.start()

2018-10-19 03:30:16 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2018-10-19 03:30:16 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
107it [00:10, 10.07it/s]2018-10-19 03:30:27 [scrapy.core.scraper] ERROR: Error processing {}
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/twisted/internet/defer.py", line 653, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/anaconda/lib/python3.6/site-packages/scrapy/pipelines/media.py", line 80, in process_item
    dlist = [self._process_request(r, info) for r in requests]
  File "/anaconda/lib/python3.6/site-packages/scrapy/pipelines/media.py", line 80, in <listcomp>
    dlist = [self._process_request(r, info) for r in requests]
  File "<ipython-input-2-137fb2e18ca2>", line 10, in get_media_requests
    file_url = item['file_urls'][0]
  File "/anaconda/lib/python3.6/site-packages/scrapy/item.py", l

IndexError
IndexError


113it [00:11, 10.01it/s]2018-10-19 03:30:28 [scrapy.core.scraper] ERROR: Error processing {}
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/twisted/internet/defer.py", line 653, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/anaconda/lib/python3.6/site-packages/scrapy/pipelines/media.py", line 80, in process_item
    dlist = [self._process_request(r, info) for r in requests]
  File "/anaconda/lib/python3.6/site-packages/scrapy/pipelines/media.py", line 80, in <listcomp>
    dlist = [self._process_request(r, info) for r in requests]
  File "<ipython-input-2-137fb2e18ca2>", line 10, in get_media_requests
    file_url = item['file_urls'][0]
  File "/anaconda/lib/python3.6/site-packages/scrapy/item.py", line 59, in __getitem__
    return self._values[key]
KeyError: 'file_urls'
2018-10-19 03:30:28 [scrapy.core.scraper] ERROR: Error processing {}
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-pac

IndexError
IndexError


117it [00:11, 10.08it/s]2018-10-19 03:30:28 [scrapy.core.scraper] ERROR: Error processing {}
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/twisted/internet/defer.py", line 653, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/anaconda/lib/python3.6/site-packages/scrapy/pipelines/media.py", line 80, in process_item
    dlist = [self._process_request(r, info) for r in requests]
  File "/anaconda/lib/python3.6/site-packages/scrapy/pipelines/media.py", line 80, in <listcomp>
    dlist = [self._process_request(r, info) for r in requests]
  File "<ipython-input-2-137fb2e18ca2>", line 10, in get_media_requests
    file_url = item['file_urls'][0]
  File "/anaconda/lib/python3.6/site-packages/scrapy/item.py", line 59, in __getitem__
    return self._values[key]
KeyError: 'file_urls'
2018-10-19 03:30:28 [scrapy.core.scraper] ERROR: Error processing {}
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-pac

...

IndexError


2018-10-19 03:36:07 [scrapy.core.scraper] ERROR: Error processing {}
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/twisted/internet/defer.py", line 653, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/anaconda/lib/python3.6/site-packages/scrapy/pipelines/media.py", line 80, in process_item
    dlist = [self._process_request(r, info) for r in requests]
  File "/anaconda/lib/python3.6/site-packages/scrapy/pipelines/media.py", line 80, in <listcomp>
    dlist = [self._process_request(r, info) for r in requests]
  File "<ipython-input-2-137fb2e18ca2>", line 10, in get_media_requests
    file_url = item['file_urls'][0]
  File "/anaconda/lib/python3.6/site-packages/scrapy/item.py", line 59, in __getitem__
    return self._values[key]
KeyError: 'file_urls'
2018-10-19 03:36:07 [scrapy.core.scraper] ERROR: Error processing {}
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/twisted/internet/d

IndexError
IndexError


1000it [06:00,  2.77it/s]2018-10-19 03:36:28 [scrapy.core.scraper] ERROR: Error processing {}
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/twisted/internet/defer.py", line 653, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/anaconda/lib/python3.6/site-packages/scrapy/pipelines/media.py", line 80, in process_item
    dlist = [self._process_request(r, info) for r in requests]
  File "/anaconda/lib/python3.6/site-packages/scrapy/pipelines/media.py", line 80, in <listcomp>
    dlist = [self._process_request(r, info) for r in requests]
  File "<ipython-input-2-137fb2e18ca2>", line 10, in get_media_requests
    file_url = item['file_urls'][0]
  File "/anaconda/lib/python3.6/site-packages/scrapy/item.py", line 59, in __getitem__
    return self._values[key]
KeyError: 'file_urls'


IndexError


...and output should be a json file in same directory as this notebook! 

In [2]:
from wand.image import Image
from PIL import Image as PI
import pytesseract
import os

In [ ]:
for each in os.list_dir('./full'):
    if '.pdf' not in each:
        continue
    
    

In [ ]:
image_pdf = Image(filename="/Users/jordangunn/Google Drive/newsrepo/newscycle/full/055bd4f85ea04b97f2cf583540bfb981ed9bce0a.pdf", resolution=300)
image_jpeg = image_pdf.convert('jpeg')

In [9]:
from pdf2image import convert_from_path
pages = convert_from_path('/Users/jordangunn/Google Drive/newsrepo/newscycle/full/055bd4f85ea04b97f2cf583540bfb981ed9bce0a.pdf', 500)
for page in pages:
    page.save('out.jpeg', 'JPEG')

In [12]:
page

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [11]:
print(pytesseract.image_to_string(PI.open('out.jpeg')))

‘-‘\I‘-'"

- ‘l

:t'3:3'Il’llit"l'lllhbbbhb’nl‘ltt.

e:"

From the authurot kaieOhl-
thc dd idtmh mull mg hit graph
that minutes animus :ur

‘Dazzlingwﬂimesisﬁhlen

Hli~htt1lilit\si\ dtmlul hinguplu is the must umlu~lwd

u mun nl llt/Jh‘lh Lulur ~. ltlt \tl tuld ll IHNH“\ int

nit n dihk l .1“ l t .lml her tumultumrs puma.“ lm- thruugh
tim tut tulm~ sum nurrugn umumt-ruhlt' lmt Main
.md st u I.” m .u latul tilm-“n II drums rude“ mm tin
glmnntuus \uarltl ni tlth huutttul ~t.tr .md tht.~ grutnt \ur
~tmltlul «nut t \t‘l’ .l‘st'ltlhlt‘d m um hunk

W will full it fascinating in iust the way theyexpecul"

,|.~Iv-

  
 

by Kitty Kelley

llh'le'i (mild iuturul \llt'rtutt \eiutmn

hlolﬁ

 

h.ulmg :ls

 

——Ptrhh~hr'r\ “(11h

—-Inlm liarklum

 

‘2 [uttt‘s nt phntnguplu

Simon and Schustcr/A

  
    
    
 

Thou. Oct. 29. 1901

 

hm ‘ “I “0-7"
“ten M %‘i&
C _ M it— “"h".
r:- htle _ l I hurt W
w M
"W enema I Della m m m mm
mm" “m" 5",”! ' rm ' W M eomo
ml! cum "I aeaha “W“ ummumm
lrmilea em I

In [14]:
import os
os.listdir('./full')

['.ipynb_checkpoints',
 '0427ac9bdd8e2b4ed939ed10d4a5acedae2e36c5.txt',
 '055bd4f85ea04b97f2cf583540bfb981ed9bce0a.pdf',
 '05a7215afa2140e1c41951ecafe6518a267cedd7.pdf',
 '07a4bc077aa17799c547bcbb9d63260b3dec5057.txt',
 '08e7906ef2fdd753289ab709b25899ddf687b589.pdf',
 '0acebcb42f106b35fe243640cb822255fa6c5784.txt',
 '0b60e833513b06fc6a0d369d581db3f852f61873.txt',
 '0df77e8aab0efe4b3b8ae5b0f6c99bd69f489c60.pdf',
 '13b1276535b1a74dae88aabc4c5e9c3d1ca8d70a.pdf',
 '13cb13e86cdf606c81a3a15e26fed8c94a777cbd.pdf',
 '13cbfc4f2691a34054baa0c31bbabc780d0729be.pdf',
 '15aeb85197144acc71aa631328f19b424a963c0b.pdf',
 '198355a5edd499405216f0ea6a1b96c9443cba37.pdf',
 '218ae0ee841ccea2fbc1cba14b63dc45def92857.pdf',
 '231d4c42dc56d8749496a605cba772b74f0312b4.pdf',
 '27c6cb5af788ab21fa9096f859f566060e19d775.txt',
 '27cca7275cce434e86a38b8ddabc49698c394fd5.pdf',
 '2920ab5bac83d551d0da1e64cfd9a20fef4c9fb5.pdf',
 '2920ec6ae7ead5b58f243ce737b8e0a6c274309c.pdf',
 '29d0eae7c4f2b617ae2bf4789a66ee12202409dd.pdf

In [6]:
input('hello?')

''

In [7]:
from urllib.request import urlopen
urlopen('http://google.com')

In [10]:
import webbrowser

url = 'http://www.google.com'

webbrowser.open_new(url)

True